# Create hosted imagery layer from local raster raster (crf)

### Docs and resources
#### https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.utils.html#arcgis.raster.utils.generate_direct_access_url
#### https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.analytics.html#add_image
#### https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.analytics.html#copy-raster

## Import needed libraries

In [ ]:
import os
import arcpy
import getpass
import arcgis
from arcgis.raster import *
from arcgis.raster.analytics import *
from arcgis.gis import GIS

arcgis.env.verbose=True
arcgis.env.overwrite = True

import urllib3
urllib3.disable_warnings() 

## Provide username and password for publishing account

In [ ]:
 from credentials import username, password, org_url

## Connect to ArcGIS Online Account

In [ ]:
gis = GIS(org_url, username, password, verify_cert=True)
gis

GIS @ https://esri.maps.arcgis.com version:2024.3

## Specify the path to the CRF file
#### parse out what is needed

In [4]:
upload_crf = r'E:\gis_lib\global_fishing_watch\v4\global_fish_watch_annual_fleet_sum_2016.crf'

### Extract the crf path, folder name, and the file name

crf_path = os.path.dirname(upload_crf)
print(crf_path)
# folder_name = "upload_crf.split('\\')[-2].replace('.crf','')"
folder_name = "imagery_upload"
print(folder_name)
crf_name = upload_crf.split('\\')[-1].replace('.crf','')
print(crf_name)

E:\gis_lib\global_fishing_watch\v4
imagery_upload
global_fish_watch_annual_fleet_sum_2016


In [5]:
crf_filename_with_extension = os.path.basename(upload_crf)
print(crf_filename_with_extension)

global_fish_watch_annual_fleet_sum_2016.crf


## generate the direct access url.
### this gives us a SAS token from Azure that allows us to decode and extract what is needed to create an ACS (Cloud Storage Connection)
### Request direct access URL from ArcGIS Online for the ArcGIS Online user that you are logged in as

In [6]:
access_url = arcgis.raster.utils.generate_direct_access_url(expiration=1440, gis=gis)
access_url

'https://stg-arcgisazureimagery.az.arcgis.com/p3eplmys2rvchkjx-8223a93974b7aa6265befa1ca28427d1?sv=2018-03-28&sr=c&sig=79O9rVBZcI7NovH0%2FMlofQ0vxTMa4UQ1ZV6DlS2ruwc%3D&st=2024-11-21T19%3A24%3A19Z&se=2024-11-22T19%3A29%3A19Z&sp=racwdl'

## parse what we need from the access_url

In [7]:
azure_account_name = access_url.split('//')[1].split('.')[0]
print("Azure account name: " + azure_account_name)
azure_container = access_url.split('?')[0].split('/')[-1]
print("Azure container name: " + azure_container)
azure_sas_token = access_url.split('?')[1].split('/')[-1]
print("Azure storage SAS Token: " + azure_sas_token)
azure_cpl_endpoint = access_url.split('/')[0] + "//" +  access_url.split('/',3)[-2]
print("Azure CPL Endpoint: " + azure_cpl_endpoint)

Azure account name: stg-arcgisazureimagery
Azure container name: p3eplmys2rvchkjx-8223a93974b7aa6265befa1ca28427d1
Azure storage SAS Token: sv=2018-03-28&sr=c&sig=79O9rVBZcI7NovH0%2FMlofQ0vxTMa4UQ1ZV6DlS2ruwc%3D&st=2024-11-21T19%3A24%3A19Z&se=2024-11-22T19%3A29%3A19Z&sp=racwdl
Azure CPL Endpoint: https://stg-arcgisazureimagery.az.arcgis.com


## Create the acs connection file locally

In [8]:
acs_folder = r'E:\gis_lib\00_common\acs_connections'
target_connection_file = "agol_azure.acs"
acs_full_path = os.path.join(acs_folder, target_connection_file)

if os.path.exists(acs_full_path):
    os.remove(acs_full_path)

# account_name = "stg-arcgisazureimagery"
account_name = azure_account_name
# container_name = "p3eplmys2rvchkjx-8223a93974b7aa6265befa1ca28427d1"
container_name = azure_container
# config_options = "AZURE_STORAGE_SAS_TOKEN sv=2018-03-28&sr=c&sig=yFsVfF%2FgEwufmbqqG7AEe5z2DF%2BOkTEtB6a5x9Rbwo8%3D&st=2024-11-20T19%3A29%3A48Z&se=2024-11-21T19%3A34%3A48Z&sp=racwdl;CPL_AZURE_ENDPOINT https://stg-arcgisazureimagery.az.arcgis.com"
config_options = "AZURE_STORAGE_SAS_TOKEN " +  azure_sas_token + ";CPL_AZURE_ENDPOINT " + azure_cpl_endpoint

source_connection = arcpy.management.CreateCloudStorageConnectionFile(
    out_folder_path=acs_folder, 
    out_name=target_connection_file,
    service_provider="AZURE", 
    bucket_name=container_name, 
    access_key_id=account_name,
    config_options=config_options)

## Transfer local files to the Azure Blob Storage

In [9]:
source = upload_crf
target = os.path.join(acs_folder, target_connection_file, folder_name, crf_name) + ".crf"
arcpy.management.TransferFiles(source, target)
print(target)
print(source)

E:\gis_lib\00_common\acs_connections\agol_azure.acs\imagery_upload\global_fish_watch_annual_fleet_sum_2016.crf
E:\gis_lib\global_fishing_watch\v4\global_fish_watch_annual_fleet_sum_2016.crf


## Direct upload image CRF to a single Tiled Imagery Layer

In [10]:
data_url = access_url.split('?')[0] + "/" + folder_name + "/" + crf_name + '.crf'
print(data_url)

# layer_name = crf_name
layer_name = crf_name.replace('_fish_','_fishing_').replace('_2016','') + "_append_test2"

ImageryLayer = copy_raster(input_raster=data_url,
                           output_name=layer_name,
                           context={"compression":"LERC 0"},
                           gis=gis)

https://stg-arcgisazureimagery.az.arcgis.com/p3eplmys2rvchkjx-8223a93974b7aa6265befa1ca28427d1/imagery_upload/global_fish_watch_annual_fleet_sum_2016.crf


c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advi

Submitted.
Executing...
Start Time: Thursday, November 21, 2024 7:36:39 PM
Hosted Imagery Privilege Check: OK
Output item id is: c64515b33f24474da15ee00850b3858c
Output image service url is: https://tiledimageservices.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/global_fishing_watch_annual_fleet_sum_append_test2/ImageServer
Output cloud raster name is: global_fishing_watch_annual_fleet_sum_append_test2
Input is a valid url https://stg-arcgisazureimagery.az.arcgis.com/p3eplmys2rvchkjx-8223a93974b7aa6265befa1ca28427d1/imagery_upload/global_fish_watch_annual_fleet_sum_2016.crf
Input raster is: https://stg-arcgisazureimagery.az.arcgis.com/p3eplmys2rvchkjx-8223a93974b7aa6265befa1ca28427d1/imagery_upload/global_fish_watch_annual_fleet_sum_2016.crf
Hosted data folder is: /cloudStores/P3ePLMYs2RVChkJx/c64515b33f24474da15ee00850b3858c
Single image: https://stg-arcgisazureimagery.az.arcgis.com/p3eplmys2rvchkjx-8223a93974b7aa6265befa1ca28427d1/imagery_upload/global_fish_watch_annual_fleet_sum

c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advi

Direct transfer single CRF.
Updating image service...
Updating service with data store URI.
Getting image service info...
Updating service: https://tiledimageservices.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/admin/services/global_fishing_watch_annual_fleet_sum_append_test2/ImageServer/edit
{'success': True}


c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advi

Portal item refreshed.


c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\keit8223\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rasteranalysis.arcgis.com'. Adding certificate verification is strongly advi

CopyRaster GP Job: bd1d832c32b943749ac0335249672be1 finished successfully.


In [12]:
ImageryLayer

<Item title:"global_fishing_watch_annual_fleet_sum_append_test2" type:Tiled Imagery Layer owner:esri_oceans>

In [ ]:
append_crf = r'E:\gis_lib\global_fishing_watch\v4\global_fish_watch_annual_fleet_sum_2017.crf'
append_crf_name = append_crf.split('\\')[-1]

print(append_crf_name)

target = os.path.join(acs_folder, target_connection_file, folder_name, append_crf_name)
arcpy.management.TransferFiles(append_crf, target)
print(target)
print(source)




global_fish_watch_annual_fleet_sum_2017
E:\gis_lib\00_common\acs_connections\agol_azure.acs\imagery_upload\global_fish_watch_annual_fleet_sum_2017.crf
E:\gis_lib\global_fishing_watch\v4\global_fish_watch_annual_fleet_sum_2016.crf


In [ ]:
# arcgis.raster.analytics.add_image(ImageryLayer, [target])


Failed to execute (AddImage).
Failed.


Submitted.
Executing...
Start Time: Thursday, November 21, 2024 8:01:24 PM
Hosted Imagery Privilege Check: OK
Input raster is: ['E:\\gis_lib\\00_common\\acs_connections\\agol_azure.acs\\imagery_upload\\global_fish_watch_annual_fleet_sum_2017.crf']
Getting image service info...
Raster type JSON is: {'rasterTypeName': None, 'rasterTypeParameters': {'averagezdem': {'url': 'https://elevation3d.arcgis.com/arcgis/rest/services/WorldElevation3D/Terrain3D/ImageServer'}}}
Raster type name is: Raster Dataset
Raster type parameters are: {'averagezdem': 'https://elevation3d.arcgis.com/arcgis/rest/services/WorldElevation3D/Terrain3D/ImageServer'}
EGDB raster store is: /rasterStores/dbraster-P3ePLMYs2RVChkJx
Getting image service info...
{'currentVersion': 11.2, 'name': 'global_fishing_watch_annual_fleet_sum_append_test2', 'serviceDescription': '', 'description': '', 'type': 'ImageServer', 'capabilities': 'Image,Metadata,TilesOnly', 'copyrightText': '', 'serviceItemId': 'c64515b33f24474da15ee00850b3

Exception: Job failed.